In [2]:
print("ok")

ok


In [3]:
%pwd

'/home/efulo/repos/ml/ragion/research'

In [4]:
import os
os.chdir("../")
%pwd

'/home/efulo/repos/ml/ragion'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#load_pdf_file function extracts data from the pdf file
def  load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents

In [7]:
extracted_data = load_pdf_file(data="data/")

In [8]:
# extracted_data

In [9]:
#text_split function splits the data into text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)

    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("length of text chunks", len(text_chunks))

length of text chunks 165


In [11]:
from langchain_huggingface import HuggingFaceEmbeddings

In [12]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

/home/efulo/anaconda3/envs/ragion/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
query_result = embeddings.embed_query("hello world")
print("length", len(query_result))

length 384


In [15]:
# query_result

In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")


In [18]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "ragion"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'x-cloud-trace-context': 'a303ed20cf30f0baece0013b5bbca9d4', 'date': 'Wed, 04 Jun 2025 08:45:47 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [19]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY


In [20]:
from langchain_pinecone import PineconeVectorStore

#embedding each chunk and upsert the embeddings into pinecone index
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [21]:
#loading existing index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [22]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [24]:
retrieved_docs = retriever.invoke("what is the minimum clearance from road base material to SHW?")

In [25]:
retrieved_docs

[Document(id='3b10967c-ef48-4fbe-a1af-4ac233323b6b', metadata={'author': 'LEA', 'creationdate': '2015-05-26T13:06:23-04:00', 'creator': 'Microsoft® Word 2013', 'moddate': '2015-05-29T10:41:16-04:00', 'page': 16.0, 'page_label': '17', 'producer': 'Microsoft® Word 2013', 'source': 'data/Stormwater_Design_Standards.pdf', 'total_pages': 54.0}, page_content='2.4.2.  MI NIM UM  GRO U ND WA TE R AN D HIG H-WA TE R CL E A RA NC E S \nAll roadways shall be designed to provide a minimum of one (1) foot between the bottom of the base course of \nthe roadway and the seasonal high groundwater table. Roadside underdrains may be used in lieu of  meeting \nthis standard, provided that the underdrains will result in the seasonal high water table under the roadway being \nlowered to the above cited level.'),
 Document(id='a08b33de-9a5c-45db-8ee3-e0fe2682401a', metadata={'author': 'LEA', 'creationdate': '2015-05-26T13:06:23-04:00', 'creator': 'Microsoft® Word 2013', 'moddate': '2015-05-29T10:41:16-04:00'

In [30]:
from langchain_openai import OpenAI
from langchain_google_genai import GoogleGenerativeAI

# llm = OpenAI(temperature=0.4, max_tokens=500)
llm = GoogleGenerativeAI(model="models/gemini-2.0-flash", google_api_key=GOOGLE_API_KEY)

In [32]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you do not know the answer, say that you "
    " do not know. Directly state the code, manual and the data source for each answer and quote the text only. always state the page number and link to the source "
    "do not add anything extra."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [33]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [34]:
response = rag_chain.invoke({"input": "what is the minimum clearance from road base material to SHW?"})
print(response["answer"])

The minimum clearance from the bottom of the road base to the seasonal high water table (SHW) is one (1) foot.

*   **Code/Manual:** "2.4.2. MI NIM UM GRO U ND WA TE R AN D HIG H-WA TE R CL E A RA NC E S"
*   **Data Source:** Provided Text
*   **Quote:** "All roadways shall be designed to provide a minimum of one (1) foot between the bottom of the base course of the roadway and the seasonal high groundwater table."


In [35]:
response = rag_chain.invoke({"input": "what is the minimum clearance from road base material to SHW for manatee county? for manatee county"})
print(response["answer"])

I am sorry, but the provided text does not contain information regarding the minimum clearance from road base material to SHW for Manatee County.


In [59]:
response = rag_chain.invoke({"input": 'what is the minimum drainage measurement for a 24 inch pipe in a residential development?'})
print(response["answer"])

This question cannot be answered from the given source.


In [29]:
response = rag_chain.invoke({"input": 'what design storm is required for a local road in manatee if it is not within a floodplain?'})
print(response["answer"])

Data Source: MANATEE COUNTY PUBLIC WORKS STANDARDS PART 2 - STORMWATER MANAGEMENT DESIGN MANUAL [05/15]  Page SW-4 
Code: C. For local streets, bridges and culverts not in the published one hundred (100) year floodplain, the design 
storm shall be twenty-five (25) year frequency. 
